## Homework 3 - Which book would you recommend?

-------------

### Importing related libraries

In [1]:
import functions

## 1. Data collection

#### 1.1. Get the list of books

* From [best books ever list](https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1) we want to collect the url associated to each book in the list and retrieve only the urls of the books listed in the first 300 pages.



* The output of this step is a `.txt` file whose single line corresponds to a book's url.

In [ ]:
#inizialize an empty list
url = []

#for each page, save the corresponding web page, find the anchor elements and save the corresponding tags  
for i in range(1, 301):
    page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+ str(i))
    soup = BeautifulSoup(page.content, features='lxml')
    tag_a = soup.find_all('a', {"class": "bookTitle"}, itemprop = "url")
    
#for each book, save the corresponding url into the array
    for j in range(0,100):
        url.append("https://www.goodreads.com"+ tag_a[j]['href'])
        
#create a txt file where for each row there is a book's url 
with open("url.txt", 'w') as f:
    f.write("\n".join(map(str, url)))


#### 1.2. Crawl books

* Download the html corresponding to each of the collected urls.


* After you collect a single page, immediatly save its `html` in a file. In this way, if your program stops, for any reason, you will not loose the data collected up to the stopping point.


* Organize the entire set of downloaded `html` pages into folders. Each folder will contain the `htmls` of the books in page 1, page 2, ... of the list of books.

# Warning :

### Do not run the below cell because it'll download over 20 GB files !
#### Before running modify the `for` loop range to a lower number (e.g. 3) 

In [ ]:
#First we open the url.txt file, reading the lines and then close the file
f = open("url.txt", "r")
lines = f.readlines()
f.close()

#Setting our parent directory and the directory associated with each page number from the list
## PLEASE CHANGE THE PARENT DIRECTORY ACCORDING TO YOUR SYSTEM
parent_dir = "C:/Users/engme/OneDrive/Desktop/ALL Materials/Data Science - Sapienza/1st Semester/ALGORITHMIC METHODS OF DATA MINING AND LABORATORY/Labs/6/Links"
directory = "Page_"
page_num = 0


#Looping for 300 times that corresponds to the number of pages
for i in range(300):
    #Incrementing page number according to each page
    page_num += 1
    #Setting the current working directory
    directory = "Page_" + str(page_num)
    #Setting the main path to create the directory
    path = os.path.join(parent_dir, directory)
    #Creating new directory
    os.makedirs(path)
    
    #Looping for 100 times, which is the number of articles per page
    for i in range(100):
        #Selecting the corresponding link
        link = lines[i][:-1]
        #Dowloading the article
        r = requests.get(link, allow_redirects=True)
        #Setting the name file to keep the track
        file_name = "article_" + str(i+1) + ".html"
        #Saving the html file to its corresponding directory
        open(parent_dir + "/" + directory + "/" + file_name, 'wb').write(r.content)

-----------

#### 1.3 Parse downloaded pages

* Extracting the books information for each book as following:

1. Title (to save as `bookTitle`)
2. Series (to save as `bookSeries`)
3. Author(s), the first box in the picture below (to save as `bookAuthors`)
4. Ratings, average stars (to save as `ratingValue`)
5. Number of givent ratings (to save as `ratingCount`)
6. Number of reviews (to save as `reviewCount`)
7. The entire plot (to save as `Plot`)
8. Number of pages (to save as `NumberofPages`)
9. Published (Publishing Date)
10. Characters
11. Setting
12. Url

In [ ]:
## Custom function designed to 
parent_dir = "C:/Users/engme/OneDrive/Desktop/ALL Materials/Data Science - Sapienza/1st Semester/ALGORITHMIC METHODS OF DATA MINING AND LABORATORY/Labs/6"

functions.info_parser(parent_dir, pages=300, tsv_articles= "tsv_articles", links= "Links", url= 'url')

All tsv files generated sucessfully in tsv_articles directory

--------------

## 2. Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the books that match the query.

First, you must pre-process all the information collected for each book by :



1. Removing stopwords
2. Removing punctuation
3. Stemming
4. Anything else you think it's needed


For this purpose, you can use the [nltk](https://www.nltk.org/) library.

In [4]:

parent_dir = "C:\\Users\\elisa\\Desktop\\Algorithmic Methods of Data Mining\\ADM-HW3\\tsv_articles\\tsv_articles"

#initialize a list that will contain the lists of tokens, one per plot 
processed_docs = []
tokenizer = functions.nltk.RegexpTokenizer(r"\w+")
stop_words = set(functions.stopwords.words('english')) 
ps = functions.PorterStemmer()

#for every article_i.tsv file, extract the Plot, tokenize it and preprocess it 
for n_art in range(1, 30001):
    directory = "article_" + str(n_art) + ".tsv"
    path = functions.os.path.join(parent_dir, directory)
    if functions.os.path.exists(path):
        plot = functions.pd.read_csv(path, delimiter = '\t', usecols = ['Plot'])
        tokens = tokenizer.tokenize(str(plot))
        processed_doc = []
        for token in tokens:
            if (token != 'Plot') & (token != '0') & (not token in stop_words):
                processed_doc.append(ps.stem(token))

        processed_docs.append(processed_doc)


In [5]:

#initialize an empty dictionary
vocabulary = {}

term_id = 1

#for every document (for every plot in our case)
for doc in processed_docs:
    
#for every token in the document
    for tok in doc:
        
#if the token is not present in the dictionary yet...
        if tok not in vocabulary:
        
#...add it and set term_id as his id
            vocabulary[tok] =  term_id
            term_id += 1
            

In [7]:

#initialize an empty dictionary
my_dict = {}

doc_id = 0
n_art= 1 
#for every document (for every plot in our case)
for doc in processed_docs:
    directory = "article_" + str(n_art) + ".tsv"
    path = functions.os.path.join(parent_dir, directory)
    if functions.os.path.exists(path):
        
    #increase the id of the document
        doc_id+=n_art
        n_tok = 0

    #for every token in the document
        for tok in doc:

    #if the id of that specific token is not present in the dictionary yet...
            if vocabulary[tok] not in my_dict:

    #...add it to the dictionary as a key and let document_doc_id be one of its values:
                my_dict[vocabulary[tok]] = ["document_"+str(doc_id)]

    #else, if this token is present in the dictionary but document_doc_id is not one of his values yet...
            elif "document_"+str(doc_id) not in my_dict.get(vocabulary[tok]):

    #append document_doc_id to his values
                my_dict[vocabulary[tok]].append("document_"+str(doc_id))
    n_art += 1


In [172]:
Search_Engine(input().split())

could


,bookTitle,Plot,Url
book_1,Dreams and Shadows,A brilliantly crafted modern tale from acclaim...,https://www.goodreads.com/book/show/39988.Matilda
book_2,The Hunger Games,"Could you survive on your own in the wild, wit...",https://www.goodreads.com/book/show/2767052-th...
